In [1]:
%run AFclass.ipynb
import ast
import itertools

gr_DG = pd.read_csv('./r/data fitting/Data/gr_Div_DG_standardized_ByNoDrug.csv', index_col='Gene_inhibition')
gr_DG.loc[:,:'S0_monoculture'][gr_DG.loc[:,:'S0_monoculture']<1e-5] = 0 

label_df = pd.read_excel(open('./Data/iML1515_GP.xlsx', 'rb'), # ? S0 rct pathway label necessary?
              sheet_name='iML1515_GP', index_col=0)
# rct_pathway_df = pd.read_csv('./Data/rct_pathway.csv')
# alpha_table = pd.read_csv("./Data/alpha_table.csv", index_col=0)
alpha_table = pd.read_csv("./Data/checkerboard_alpha_table.csv", index_col=0)
if len(alpha_table.columns) == 3:
    alpha_table.columns = ['E0', 'S0.ac', 'S0.glc']
else:
    alpha_table.lv_pairs = alpha_table.lv_pairs.apply(ast.literal_eval)

comb_list = list(pd.read_csv('./Data/gr_Div_DG_Blis_Aug31.csv').gene_inhibition[1:])
DG_list = list(gr_DG.index)
if 'Normal' in DG_list:
    DG_list.remove('Normal')

def convert_arg_to_list(arg):
    if isinstance(arg, (pd.Series, pd.Index)):
        arg = list(arg) 
    elif not isinstance(arg, (list, tuple, set)):
        arg = [arg] # differ from list(arg) -> conversion of str
    return arg

def get_Biomass_df(files):
    return pd.concat(
            [pd.read_csv(file, index_col='cycle')
             for file in convert_arg_to_list(files)]
        ,axis=1)

# Biomass_df = get_Biomass_df(['./Data/BM_SG1.csv', './Data/BM_DG1.csv'])
Biomass_df = get_Biomass_df(['C:/Users/wongt/OneDrive/COMETS_Jupyter/Run_Comets/r/data fitting/Data/checkerboard_run3.csv'])

C:\Users\wongt\AppData\Roaming\Python\Python311\site-packages\nbformat\__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [338]:
log_step=5

def search_gr_cycle_with_biomass(df_search, biomass_values):
    return [df_search[df_search >= biomass_value].idxmin() 
                for biomass_value in list(biomass_values)]

def get_maximum_growth_cycle(desired_BM):
    c_max_gr = desired_BM.iloc[1]+ (desired_BM.iloc[-1] - desired_BM.iloc[1])/2
    bool_growing = ((desired_BM.iloc[-1]-desired_BM.iloc[-5])/desired_BM.iloc[-1]).apply(lambda x: x > 1e-10)
    for k, bool_grow in bool_growing.items():
        if bool_grow:
            c_max_gr[k] = desired_BM[k].iloc[-6]
    biomass_diff = (desired_BM.iloc[-1]-desired_BM.iloc[0])
    start = desired_BM.iloc[0] + biomass_diff*0.1
    end = desired_BM.iloc[0] + biomass_diff*0.9
    return c_max_gr, start, end, bool_growing

# def find_down_neighbour(df):
#     max_gr = get_cycle_max_gr(df)
#     position = list()
    
#     def correct_cycle(cycle):
#         mod = cycle%log_step
# #         return cycle if mod==0 else cycle+log_step-mod # round up
#         return cycle if mod==0 else cycle-mod # round down
# #   
#     gr_cycle = defaultdict(dict)
#     for gcomb, biomass_value in max_gr.items():
#         df_search = df.loc[:,gcomb]
#         upperneighbour_ind =  search_gr_cycle_with_biomass(df_search, biomass_value) # 0 if growth belolw 1e-8 
# #         gr_cycle[gcomb] = correct_cycle(upperneighbour_ind) 
#         gr_cycle[gcomb].update({'cycle': correct_cycle(upperneighbour_ind),
#                                 'growth phase': [search_gr_cycle_with_biomass(),
#                                                 search_gr_cycle_with_biomass()]})
#     return gr_cycle

# def get_desired_cycle_df():
#     desired_cycle_dict = defaultdict(dict)
#     for k, cycle in find_down_neighbour(Biomass_df).items():
#         key_items = k.split('_')
#         gcomb = key_items.pop(1)
#         col_name = '_'.join(key_items)
#         if desired_cycle_dict[gcomb].get(col_name):
#             sys.exit("Halt, Gene comb keys get overwritten")
#         desired_cycle_dict[gcomb].update({col_name: cycle})

#     return pd.DataFrame.from_dict(desired_cycle_dict, orient='index')


def get_desired_cycle(Biomass_df, log_step=5):
    def correct_cycle(cycle): # 
        if cycle < log_step:
            return log_step
        return round(cycle / log_step) * log_step

    def get_growth_phase_length():
        return ((desired_cycle['end'] - desired_cycle['start'])*(1-desired_cycle.bool_growing) + # if not growing, not changing growth phase length
                1e4*(desired_cycle.bool_growing)) #if growing, set growth length to 999
    
    def split_index_to_cols(df):
        items = df.index.str.split('_')
        columns = ['Species', 'Gene_inhibition', 'culture']
        if len(items[0]) > 3:
            columns.extend(['alpha_lv_pairs'])
        return pd.DataFrame(items.tolist(), index=df.index, columns=columns)
    desired_biomass_df = pd.DataFrame(get_maximum_growth_cycle(Biomass_df), index=['c_max_gr', 'start', 'end', 'bool_growing'])
    
    desired_cycle = (desired_biomass_df.iloc[:-1]
                .apply(lambda x: 
                        search_gr_cycle_with_biomass(Biomass_df.loc[:,x.name],x))
                .T)
    desired_cycle['bool_growing'] = desired_biomass_df.T.bool_growing
    desired_cycle['cycle_max_gr'] = desired_cycle['c_max_gr'].apply(correct_cycle) # -> cycle_max_gr
    desired_cycle['growth_phase'] = desired_cycle[['start', 'end']].values.tolist()
    desired_cycle['growth_phase_length'] = get_growth_phase_length()
    desired_cycle['end_cycle'] = Biomass_df.index[-1]//5*5
    desired_cycle = desired_cycle.join(split_index_to_cols(desired_cycle))
#     .query('culture=="coculture"')
    if len(desired_cycle.Gene_inhibition.unique()) >1:
        desired_cycle = desired_cycle.set_index('Gene_inhibition')[['cycle_max_gr', 'bool_growing', 'growth_phase','growth_phase_length', 'Species','culture','end_cycle']]
    else:
        desired_cycle.index = ['_'.join([x[1],x[3]]) for x in desired_cycle.index.str.split('_')]
        desired_cycle.Gene_inhibition = desired_cycle.index

    return desired_cycle     

In [339]:
desired_cycle = get_desired_cycle(Biomass_df, log_step=10)


# extract flux at maximum growth cycle


In [69]:
def get_gene_id(model, gene_name):
    for i in model.genes:
        if(i.name == gene_name):
            return(i.id)

def get_rcts_list(model, gcomb_list): 
    rcts_list = list()
    rcts_set = set()
    for i, gene in enumerate(gcomb_list):
        gene_rcts = [rct.id for rct in model.genes.get_by_id(get_gene_id(model, gene)).reactions]
        if i > 0:
            gene_rcts = list(set(gene_rcts) - rcts_set)
        rcts_list.append(gene_rcts)
        rcts_set = rcts_set | set(gene_rcts)
    return rcts_list

def adjust_flux_df(model, df, gene_combo: list, alpha_table=alpha_table): 
    # Don't use v1 cols to indicate gene_inhibition, will skip unidirectional reaction
    def query_alpha(gene_combo, alpha_table):
        splitted = gene_combo.split('.')
        if len(splitted) == 2:
            gcomb_alpha = {gene: alpha_table.loc[gene, f'{model.id}'] for gene in gene_combo.split('.')}
            # return gcomb_alpha

        # alphas
        splitted = gene_combo.split('_')
        Gene_inhibition, lv_pair = [ele.split('.') for ele in 'folP.folA_1.2'.split('_')]
        lv_pair = tuple([int(ele) for ele in lv_pair])
        gcomb_alpha = dict()
        alpha_table = alpha_table.query('lv_pairs == @lv_pair')
        for current_gene in Gene_inhibition:
            # print(alpha_table.head())
            gcomb_alpha.update({current_gene: 
                                alpha_table.loc[current_gene, f'{model.id}']})
        return gcomb_alpha, alpha_table

    print(gene_combo)


    if 'Normal' not in gene_combo:
#         gene_combo_dict = get_gcomb_alpha_dict(gene_combo) 
        v1_cols = df.filter(regex='v1').columns
        
        # gcomb_alpha = {gene: alpha_table.loc[gene, f'{model.id}'] for gene in gene_combo.split('.')}
        gcomb_alpha, alpha_table = query_alpha(gene_combo, alpha_table)
        # print(gcomb_alpha)
        gcomb_alpha = dict(sorted(gcomb_alpha.items(), key=lambda item: item[1], reverse=True))
        
        rcts_list = get_rcts_list(model, gcomb_alpha.keys()) # exclude repeated rct for two gene
        scaled_rcts = list()

        for gene, rcts in zip(gcomb_alpha.keys(), rcts_list):  
#             rcts = [rct for rct in rcts if rct in orig_cols]
            for orig_col in rcts:
                alpha = alpha_table.loc[f'{gene}', f'{model.id}']    
                v1_col = orig_col + "_v1"
                reversible = v1_col  in v1_cols
                if reversible: 
                    df[f'{orig_col}'] = (df[f'{orig_col}'] + df[f'{v1_col}'])/alpha # only forward or backward != 0  
                    df = df.drop(f'{v1_col}', axis=1)  
                else:
                    df[f'{orig_col}'] = (df[f'{orig_col}'])/alpha # only forward or backward != 0              
    return df

def get_desired_BM(Biomass_df = Biomass_df, regex='Normal_coculture'):
    return Biomass_df.filter(regex=regex)
  
def supply_line(file):
#     file = f'./Data/{file}'
    with open(file) as fh:
        for line in fh:
            yield json.loads(line) 

def retrive_specific_culture(file_list: list, culture_item: str, XG = 'DG'): # culture items:'coculture_media', 'E0_coculture_flux', 'S0_ac_coculture_flux', 'E0_monoculture_media', 'E0_monoculture_flux', 'S0_ac_monoculture_media', 'S0_ac_monoculture_flux'
    def modify_culture_dict_keys(k: str, XG: str):
        if k!='coculture_media':
            k_list = k.split('_')
            k_list[:-2] =  ['.'.join(k_list[:-2])]
            k_list.append(XG)
        else: 
            k_list = [k, XG]
        return '_'.join(k_list)

    flux_dict = dict()
    coculture_dict = dict()
    file_list, XG = convert_arg_to_list(file_list), convert_arg_to_list(XG)

    for file, current_XG in itertools.zip_longest(file_list, XG, fillvalue=None):
        if file is not None: # get SG DG flux from sasme json file
            supp = supply_line(file)
           
        # prefix in json 
        culture_items = [f'E0_{culture_item}', f'S0_{culture_item}'] if culture_item != 'coculture_media' else [culture_item]
        for line in supp:    # culture as column <=> keys
    #         coculture_dict = line.get(f'{culture_item}')
            if any([ele in line.keys() for ele in culture_items]):
                coculture_dict.update(
                    {modify_culture_dict_keys(k, current_XG): v 
                     for k, v in line.items() if k in culture_items}) 
    return coculture_dict # {E0_coculture: Dict, S0_coculture: Dict}

# pwy group with more than 2 element
def get_pwypair_gp_GRT2(p_o: dict): # index greater than 2 element
    flux_compare_combos_dict = {k:v for k,v in p_o.items() if len(v)>=2}
    return flux_compare_combos_dict

def remove_Zero_col(df): # extend N differ than 0 
    return(df.loc[:, ((df !=0) & (df.notnull())).any(axis=0)]) # ignore NA entry 

def clean_flux_df(df):
    return remove_Zero_col(df.dropna())

def get_XG_cycle_from(desired_cycle):

    if len(desired_cycle.index[-1].split('.')) <=2:
        SG_cycle = desired_cycle.loc[[len(ele.split('.')) ==1 for ele in desired_cycle.index]]
        DG_cycle = desired_cycle.loc[[len(ele.split('.')) ==2 for ele in desired_cycle.index]]
    else:
        SG_cycle = desired_cycle.loc[['0' in ele for ele in desired_cycle.index]]
        DG_cycle = desired_cycle.loc[['0' not in ele for ele in desired_cycle.index]]
    SG_cycle.index.name='SG'
    DG_cycle.index.name='DG'
    SG_cycle.columns.name=None
    DG_cycle.columns.name=None
    return SG_cycle, DG_cycle    

# def get_SG_DG(explode=True):
#     df = pd.DataFrame(pd.DataFrame(2*[DG_list], index=['Gene_inhibition','SG']).T.set_index('Gene_inhibition').SG.str.split('.'))
#     return df.explode('SG') if explode else df
   
# def get_alpha_wide(alpha_table):
#     S = [ele for ele in alpha_table.columns if 'S0' in ele]
#     return alpha_table.melt(value_vars=[E0.id, S[0]], var_name='Species', value_name='alpha', ignore_index=False)

# def get_alpha_to_merge(alpha_table):
#     def get_alpha(l: list[str, str], alpha_table):
#         l=l[0]
#     #     print(l)
#         Species = [E0.id, S0.id]
#         df = pd.DataFrame(alpha_table.loc[l, Species]).T
#         return df.values.tolist()
#     # a.apply(get_alpha, axis=1, result_type='broadcast')

#     S = [ele for ele in alpha_table.columns if 'S0' in ele]
#     ES_cols = ['E0', S[0]]
#     alpha_wide = get_alpha_wide(alpha_table)
#     df = (get_SG_DG(explode=False)
#           .apply(lambda x: pd.Series(get_alpha(x, alpha_table), index=ES_cols), axis=1) # df with E0 S0 alpha columns
#           .melt(value_vars=ES_cols, var_name='Species', value_name='alpha', ignore_index=False)
#           .combine_first(alpha_wide))
#     return df
# alpha_to_merge = get_alpha_to_merge(alpha_table)

In [342]:
def get_growth_phase_summary(model, culture, df_from_js, desired_cycle, current_gene_combo, is_flux=True):
    
    growth_phase = desired_cycle.query('Species == @model.id & culture == @culture').loc[current_gene_combo, 'growth_phase']
    print(growth_phase)
    growth_df = df_from_js.query('cycle>@growth_phase[0] & cycle<@growth_phase[1]').drop('cycle', axis=1)
#     growth_df = growth_df.drop(['cycle','x','y'], axis=1) if is_flux else growth_df.drop(['cycle','x','y'])
    summary_df = (growth_df.apply(['mean','std'])
         .unstack().to_frame().T)
    summary_df.columns = summary_df.columns.map('_'.join).str.strip('_')
    summary_df.index= [current_gene_combo]
    return summary_df

def retrive_specific_keys(model, file, culture_item, desired_cycle, desired_cycle_col='cycle_max_gr',alpha_table=alpha_table):
    # ?convert to apply function speed up?
    # INDEX in desired_cycle determines SG/DG list to query 
    def get_flux_dict(coculture_dict, model, desired_cycle, desired_cycle_col=desired_cycle_col):
#         if model.id in coculture_dict.keys(): # ?delete?
#             coculture_dict = coculture_dict[model.id]x
        flux_dict, no_grow = dict(), list()
        culture = culture_item.split('_')[0]
#         item_is_flux = culture_item.split('_')[1] == 'flux'
#         for current_gene_combo, cycle in desired_cycle[[f'{model.id}_{culture}']].iterrows():
#         print(desired_cycle.query('Species == @model.id & culture == @culture').index)
        for current_gene_combo, cycle_row in desired_cycle.query('Species == @model.id & culture == @culture').iterrows():
            # print(current_gene_combo, cycle_row)
            
            cycle = cycle_row[desired_cycle_col] # iterrow create subdata frame of one column 
            
            print('ccyc', cycle)
            if cycle < 10:
                no_grow.append(current_gene_combo)
                flux_dict[current_gene_combo] = pd.DataFrame(index = [f'{current_gene_combo}'])
            else:
                desired_js = coculture_dict.get(f'{current_gene_combo}') if type(coculture_dict) is dict else coculture_dict[current_gene_combo]
                # coculture_dict['E0_monoculture_flux_SG']['folP.folA_1.0']
                # df_from_js = pd.read_json(desired_js)
                df_from_js = pd.DataFrame.from_dict(desired_js)
                is_flux = '_flux' in culture_item
                if is_flux:
                    df_from_js = adjust_flux_df(model, df_from_js, current_gene_combo,alpha_table).drop(['x','y'], axis=1)
                    
                else:
                    df_from_js = df_from_js.pivot(index=['cycle'],columns=['metabolite'], values='conc_mmol').reset_index()
                growth_phase_summary = get_growth_phase_summary(model, culture, df_from_js, desired_cycle, current_gene_combo, is_flux=is_flux)
                
                temp_df = df_from_js.query("cycle == @cycle")

                print(temp_df.cycle, current_gene_combo, cycle_row, cycle)

#                 if '_media' in culture_item:
#                     temp_df = temp_df.set_index(['metabolite','cycle']).drop(['x','y'],axis=1).unstack([0]).conc_mmol.reset_index() # media stored as stacked form 
                temp_df.index = [current_gene_combo]
        #                 return temp_df, current_gene_combo
                temp_df = pd.concat([temp_df, growth_phase_summary], axis=1) # mean, sd for each reaction -> ncols*3
                
                flux_dict[current_gene_combo] = temp_df # df as 
        #             genes_dict = {current_gene_combo: genes_dict.get(f'{current_gene_combo}') for current_gene_combo in genes_dict}  #subset of gene
        if no_grow:
            print('Zero growth: ', ', '.join(no_grow))
        return pd.concat(flux_dict.values()).copy() # pd.read_json() culture_item -> col pd.read_json()
#         return pd.concat(flux_dict.values()) # pd.read_json() culture_item -> col pd.read_json()
    
#     print(type(file) is dict)
    if type(file) in [dict, pd.DataFrame, pd.Series]:
        return get_flux_dict(file, model, desired_cycle)
    else: # list of filename, str filename
        coculture_dict = retrive_specific_culture(file, culture_item)
        return get_flux_dict(coculture_dict, model, desired_cycle) 
    
def set_GI_SP_as_MI(end_BM, manual_SI = False, multi_index = None):
    if type(end_BM.index[0]) is not int: # Gene_inhibition is not index # Series.dtype -> dtype('int64')
        if end_BM.index.name in [None, 'DG']:
            end_BM.index.name = 'Gene_inhibition'
        if end_BM.index.name == 'Gene_inhibition':
            if 'Gene_inhibition' in end_BM.columns:
                end_BM = end_BM.drop('Gene_inhibition', axis=1)
            end_BM = end_BM.reset_index()
    if multi_index:
        return end_BM.set_index(multi_index)
    if 'Species' in end_BM.columns and manual_SI == False:
        return end_BM.set_index(['Gene_inhibition', 'Species'])
    else:
        return end_BM.set_index('Gene_inhibition')

# def join_dfs_using_MI(df1, df2, how='left'):
#     return set_GI_SP_as_MI(df1).join(set_GI_SP_as_MI(df2), how=how)

def join_dfs_using_MI(df_list, how='left', multi_index = None):
    df_list = [set_GI_SP_as_MI(df, multi_index=multi_index) for df in df_list]
    result_df = df_list[0]
    for df in df_list[1:]:
        result_df = result_df.join(df, how=how)
    return result_df

def get_flux_compare_df(culture_item, coculture_dict=None, desired_cycle_col='cycle_max_gr', desired_cycle = desired_cycle, alpha_table=alpha_table): # also work for extracting metabolite
    def get_key(current_species, culture_item, current_XG):
        if culture_item != 'coculture_media':
            return '_'.join([current_species, culture_item, current_XG]) 
        return '_'.join([culture_item, current_XG]) 
    
    file_list = ['./Data/flanalysis_BM_SG.json', './Data/flanalysis_BM_DG.json'] 
    if coculture_dict is None:
        coculture_dict = retrive_specific_culture(file_list, culture_item=culture_item, XG=['SG','DG']) 
    flux_dict = dict()
    for species, XG_cycle in itertools.product([E0, S0],[DG_cycle, SG_cycle]):
        current_XG = XG_cycle.index.name
        current_species = species.id
        print(current_species, current_XG)
        key = get_key(current_species, culture_item, current_XG) 
        if key not in flux_dict.keys(): # prevent repeated extraction of df for 'coculture_media'
            flux_df = retrive_specific_keys(species, coculture_dict[key], culture_item, XG_cycle, desired_cycle_col=desired_cycle_col, alpha_table=alpha_table)
            flux_df['Species'] = current_species
            flux_df['XG'] = current_XG
            flux_dict[key] = flux_df
    # return flux_dict
    flux_compare_df = pd.concat(flux_dict.values()).copy()
    flux_compare_df.index.name = 'Gene_inhibition'
#     flux_compare_df = get_ESdiff(remove_Zero_col(flux_compare_df)) # ES diff 41 columns ?only common
#     flux_compare_df = get_Ndiff(flux_compare_df)
    if 'lv_pairs' not in alpha_table.columns: #  alpha value not joined for checkerboard, SG format different 
        flux_compare_df = join_dfs_using_MI(get_alpha_to_merge(alpha_table), flux_compare_df, how='right') # DG only
    flux_compare_df['culture'] =  culture_item.split('_')[0]
    return flux_compare_df

def get_SG_spec_flux(gene_list):
    mono_dict = retrive_specific_culture(file_list, culture_item='monoculture_flux', XG=['SG','DG'])
    Eflux_spec = retrive_specific_keys(E0, mono_dict['E0_monoculture_flux_SG'], 'monoculture_flux', desired_cycle.loc[gene_list])
    Sflux_spec = retrive_specific_keys(S0, mono_dict['S0_monoculture_flux_SG'], 'monoculture_flux', desired_cycle.loc[gene_list])
    return Eflux_spec, Sflux_spec

def get_flux_spec(gene_list='dadX', models=[E0, S0], culture_item='coculture_flux',XG='SG'):
    gene_list=convert_arg_to_list(gene_list)
    result_df = pd.DataFrame()
    for model in models:
        flux_spec = pd.concat([
            pd.DataFrame([f'{model.id}']*len(gene_list), index=gene_list, columns=['Species']),
            retrive_specific_keys(model, coculture_dict[f'{model.id}_{culture_item}_{XG}'], culture_item, desired_cycle.loc[gene_list])
        ],axis=1)
        result_df = pd.concat([result_df, flux_spec])
    return result_df

def get_ICX_from_lv_pair_list(lv_pair_list=[4.2, 5.3]):
    ICX = dict()
    corrected_lv_pair_list = list()
    for lv_pair in convert_arg_to_list(lv_pair_list):
        lv_pair = str(lv_pair).split('.')
        lv_pair = tuple([int(x) for x in lv_pair])
        corrected_lv_pair_list.append(lv_pair)
        # print(alpha_table.query('lv_pairs==@lv_pair').ICX.unique())
        # print(lv_pair)
        ICX.update({lv_pair: alpha_table.query('lv_pairs==@lv_pair').ICX.to_dict()})
    return ICX 

In [ ]:
SG_cycle, DG_cycle = get_XG_cycle_from(desired_cycle)
SG_cycle
file_list = ['C:/Users/wongt/Downloads/flanalysis_checkerboard_run3.json']
# coculture_dict = retrive_specific_culture(file_list, culture_item='coculture_flux', XG=['SG','DG'])
# mono_dict = retrive_specific_culture(file_list, culture_item='monoculture_flux', XG=['SG','DG'])

co_flux_compare_df = get_flux_compare_df('coculture_flux', coculture_dict, desired_cycle=desired_cycle, alpha_table=alpha_table) 
mono_flux_compare_df = get_flux_compare_df('monoculture_flux', mono_dict, desired_cycle=desired_cycle, alpha_table=alpha_table) 

In [344]:
lv_pairs = [lv_pair[1].split('.') for lv_pair in flux_compare_df.index.str.split('_')]
lv_pairs = [tuple([int(ele) for ele in lv_pair]) for lv_pair in lv_pairs]
co_flux_compare_df['lv_pairs'] = lv_pairs
# arrange 'Species', 'XG', 'lv_pairs' to front
cols = co_flux_compare_df.columns.tolist()
cols = ['Species', 'XG', 'lv_pairs'] + [col for col in cols if col not in ['Species', 'XG', 'lv_pairs']]
flux_compare_df = co_flux_compare_df[cols]

In [12]:
# desired_cycle.query('culture=="coculture"').merge(flux_compare_df,
#                                                   left_index=True, right_index=True)

flux_compare_df = pd.concat([flux_compare_df, mono_flux_compare_df])
flux_compare_df = join_dfs_using_MI([desired_cycle, flux_compare_df], multi_index=['XG','Gene_inhibition', 'culture','Species'])
flux_compare_df['ICX'] = flux_compare_df.apply(lambda x : get_ICX_from_lv_pair_list(x.alpha_lv_pairs).values(), axis=1)
# flux_compare_df.to_csv('./Data/checkerboard_flux_compare_df.csv')
# flux_compare_df.loc[lv_pair_list][['growth_phase','EX_met__L_e','EX_lcts_e','EX_gal_e','EX_ac_e','EX_ac_e_mean','EX_ac_e_std','EX_co2_e',
#                                                                      'DHPS2','DHFR',  'BIOMASS_Ec_iML1515_core_75p37M','BIOMASS_iRR1083_metals']]


# Maintainance flux

In [328]:
maintaince_flux = coculture_dict['E0_coculture_flux_SG']['folP.folA_5.5']

# Possible single gene format for checkerboard

In [131]:
def get_checkerboard_inhibition_SG(gene, lv_pairs):
    # print(gene, lv_pairs)
    mask = [gene == g for g in 'folP.folA'.split('.')]
    if isinstance(lv_pairs, str):
        lv_pairs =ast.literal_eval(lv_pairs)
        
    return ''.join([gene+'_'+str(x) for x, m in zip(lv_pairs, mask) if m]) # should be only element 

alpha_table.reset_index().apply(lambda x: get_checkerboard_inhibition_SG(x.Gene_inhibition, x.lv_pairs), axis=1)

0     folP_0
1     folA_0
2     folP_0
3     folA_1
4     folP_0
       ...  
67    folA_3
68    folP_5
69    folA_4
70    folP_5
71    folA_5
Length: 72, dtype: object